In [18]:
import numpy as np
import os
import re
import scipy.io as scio
import numpy as np
import pandas as pd
import scipy.signal
from sklearn.preprocessing import LabelBinarizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [19]:
raw_num = 240
col_num = 2000

class Data(object):
    
    def __init__(self):
        self.data = self.get_data()
        self.label = self.get_label()

    def file_list(self):
        return os.listdir('C:/Users/KQiao/Desktop/AMCNN-BiGRU/data483/')
    
    def get_data(self):
        file_list = self.file_list()
        for i in range(len(file_list)):
            file = scio.loadmat('C:/Users/KQiao/Desktop/AMCNN-BiGRU/data483/{}'.format(file_list[i]))
            for k in file.keys(): # 获取 DE 相关数据
                file_matched = re.match('X\d{3}_DE_time', k)
                if file_matched:
                    key = file_matched.group()
            if i == 0:
                data = np.array(file[key][0:480000].reshape(raw_num,col_num))
            else:
                data = np.vstack((data, file[key][0:480000].reshape(raw_num,col_num))) # 按行堆叠
        return data
    
    def get_label(self):
        file_list = self.file_list()
        title = np.array([i.replace('.mat', '') for i in file_list]) # 用文件名作为 标签
        label = title[:, np.newaxis] # 增加一个维度
        label_copy = np.copy(label)
        for _ in range(raw_num-1):
            label = np.hstack((label, label_copy))
        return label.flatten()

In [20]:
Data = Data()
data = Data.data # (2400, 2000)
label = Data.label # (2400,)
lb = LabelBinarizer()
y = lb.fit_transform(label) # one-hot 编码

# Wiener filtering 主要用于从带噪的观测信号种提取出所需要的干净信号
data_wiener = scipy.signal.wiener(data, mysize=3, noise=None)

print(data.shape)
print(y.shape)

# downsampling 下采样，每隔7个点取一个值
index = np.arange(0,2000, 8)
data_samp = data_wiener[:, index]
print(data_samp.shape)

(2400, 2000)
(2400, 10)
(2400, 250)


In [21]:
X_train, X_testss, y_train, y_testss = train_test_split(data_samp, y, test_size=0.3, shuffle=True)
X_test, X_valid, y_test, y_valid = train_test_split(X_testss, y_testss, test_size=0.5, shuffle=True)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(1680, 250) (360, 250) (360, 250) (1680, 10) (360, 10) (360, 10)


In [22]:
# 用于保存划分的数据集
scio.savemat('traindata.mat',{'data': X_train, 'label': y_train})
scio.savemat('validdata.mat',{'data': X_valid, 'label': y_valid})
scio.savemat('testdata.mat',{'data': X_test, 'label': y_test})